# Commands for NGS project

# Preprocessing

Prefix for nice

In [ ]:
nice -n 19

Using fastqc to get an anlaysis to the Fastq file

In [ ]:
parallel fastqc ::: [INPUT FASTQ GZ FILE 1] [INPUT FASTQ GZ FILE 2]

Filtering data according to different filters using trimmomatic

Option | Effect
------ | ------
ILLUMINACLIP   | 	The sequence of the adapters, the remaining numbers are sensitivity thresholds (see software manual for the exact definitions)
  LEADING      | Remove the bases at the 5' end when the QC scores fall below this threshold.
  TRAILING    | Remove the bases at the 3' end when the QC scores fall below this threshold.
  SLIDINGWINDOW | Remove bases using a sliding window strategy, the first number is the window size, the second number is the quality score.
  MINLEN | minimum length for a sequence.


In [ ]:
java -jar /home/ctools/Trimmomatic-0.39/trimmomatic-0.39.jar SE -threads 1 -phred33  \
[INPUT FASTQ GZ] [OUTPUT FASTQ GZ]  ILLUMINACLIP:/usr/share/trimmomatic/TruSeq2-PE.fa:2:30:10  \
LEADING:15 TRAILING:15 SLIDINGWINDOW:5:15 MINLEN:50

# Alignment

Align to the reference genome (for multiplexing we can add the -R option)

In [ ]:
bwa index [REF FASTA] # only once to create bwa index
bwa mem (-R "@RG\tID:RG38\tSM:SMPL20") [REF FASTA]  [INPUT FASTQ GZ] | samtools view -bS > [OUTPUT BAM]
# Sorting and indexing at same time
samtools sort --threads=4 --write-index [OUTPUT BAM] -o [OUTPUT SORTED BAM]

Merging bam file (optional) and writing index

In [ ]:
samtools merge -c --write-index [OUTPUT SORTED BAM] [INPUT SORTED BAM 1] [INPUT SORTED BAM 2]

Read Counting

In [ ]:
featureCounts -T 16 -a <annotation_file> -o <output_file>.tsv input_file_sample1.bam input_file_sample2.bam ...

Converting to count matrix

In [ ]:
cat featureCountOutput.tsv | sed 1d | cut -f1,7- | less -S | sed -e 's\header_name_old\head_name_new' ... > CountMatrix.tsv

Annotating the genes --> Find a way (maybe ask)

In [ ]:
bedtools intersect -a [INPUT GFF FILE] -b [INPUT BAM FILE] > [OUTPUT OVERLAP]

# Pseudo allignment

-l and -s are estimated average length fragment and estimated standard deviation (required for single end)

In [ ]:
# To estimate -s and -l
zcat [INPUT FASTQ GZ] | awk '{if(NR%4==2) print length($1)}' | sort -n | uniq -c > [OUTPUT TXT] # Counting the different read lengths
# Returning the SD and MEAN (you can pipe the two commands)
cat [OUTPUT TXT] | awk '{sum += $1 * $2; sumsq += $1 * $2 * $2; freq += $1} END {print "Mean:", sum / freq; print "Standard deviation:", sqrt (sumsq / freq - (sum / freq)^2)}'

In [ ]:
# First index (pseudoalignment) with reference transcriptome FASTA file
/home/ctools/kallisto/build/src/kallisto index -i [OUTPUT INDEX] [FASTA FILE]
# Use kallisto to quantify
/home/ctools/kallisto/build/src/kallisto quant -i [OUTPUT INDEX] -o [OUTPUT RDATA] --single -l 2 -s 76 [FASTQ GZ INPUT]